Up to now (05.05.2022) not further processed.

In [1]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# -*- coding: utf-8 -*-
# import dataiku
import pandas as pd, numpy as np
# from dataiku import pandasutils as pdu
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from datetime import timedelta
import math
import time

In [ ]:
"""
Get the commentaries of each new articles.
"""

# Read recipe inputs
total = dataiku.Dataset("20min_subs_totals")
total_df = total.get_dataframe()

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Configure webdriver
opts = FirefoxOptions()
opts.add_argument("--headless")
driver = webdriver.Firefox(options=opts)
driver.implicitly_wait(4)

#get the date of the most recent article
total_df = total_df.sort_values(by='published')
most_recent_article = total_df.iloc[-1]['published']

#get the ids of the articles that are less than 15 days old
recent_date = pd.to_datetime(most_recent_article-timedelta(15), utc=True)
mask = (total_df['published'] > recent_date) & (total_df['is_com_or_sub'] == 'sub')
recent_articles = total_df.loc[mask]
articles_ids = recent_articles.id.to_list()

last_com_check = dataiku.get_custom_variables()["20min_last_check_coms"]
last_com_check = pd.to_datetime(last_com_check, format='%Y-%m-%d %H:%M:%S+00:00', utc=True)


In [ ]:
new_coms = pd.DataFrame(columns=['author', 'published', 'summary', 'id', 'is_com_or_sub', 'link'])
# get the commentaries on those articles
for article_id in articles_ids:
    url = "https://www.20min.ch/comment/"
    comments_url = url + str(article_id)
    driver.get(comments_url)

    # scroll to load entire page (because it uses lazy loading)
    check_height = driver.execute_script("return document.body.scrollHeight;") 
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        height = driver.execute_script("return document.body.scrollHeight;") 
        if height == check_height: 
            break 
        check_height = height

    # get authors
    authors = driver.find_elements_by_class_name("CommentCard_authorNickname__Bpo6l")
    author_list = [author.get_attribute('innerHTML') for author in authors]
    # get publication dates
    published_date = driver.find_elements_by_class_name("CommentCard_createdAt__1ilEx")
    date_list = [pd.to_datetime(date.get_attribute('innerHTML'), format='%d.%m.%Y, %H:%M', utc=True) for date in published_date]
    print(len(date_list))
    print(date_list)
    # get text bodys
    bodys = driver.find_elements_by_class_name("CommentCard_body__1f7-z")
    body_list = [body.get_attribute('innerHTML') for body in bodys]

    # assesrt to make sure that we have all the data for every com
    # may fail if the webdriver didn't wait for the page to load
    assert len(body_list) == len(date_list) == len(author_list), "Assertion problem, probably due to page parsing before page is fully loaded"

    temp_df = pd.DataFrame(list(zip(author_list, date_list, body_list)), columns =['author', 'published', 'summary'])
    temp_df['id'] = article_id
    temp_df['is_com_or_sub'] = 'com'
    temp_df['link'] = comments_url
    temp_df = temp_df[temp_df['published'] > last_com_check]
    print(f'Article {comments_url} has {len(temp_df)} comments')
    if len(temp_df) >0:
        new_coms = new_coms.append(temp_df, ignore_index=True)

driver.quit()

In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
if len(new_coms) > 0:
    new_last_check_coms = new_coms['published'].max()
    project_handle = dataiku.api_client().get_project(dataiku.default_project_key())
    variables = project_handle.get_variables()
    variables['standard']['20min_last_check_coms'] = str(new_last_check_coms)
    project_handle.set_variables(variables)

In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
liverss_comments_df = new_coms # For this sample code, simply copy input to output


In [ ]:
# Write recipe outputs
liverss_comments = dataiku.Dataset("20min_liverss_comments")
liverss_comments.write_with_schema(liverss_comments_df)